In [4]:
from pathlib import Path

In [5]:
sample = Path('crop')
labels = Path('result/exp2/labels')

In [7]:
str(Path.cwd())

'/home/astra/projects/hackathon/omsk_chempionat/model-predict'

In [1]:
!python yolov5/detect.py --weights ../best.pt --source crop/ --imgsz 256 256 --save-txt --project result/

detect: weights=['../best.pt'], source=crop/, data=yolov5/data/coco128.yaml, imgsz=[256, 256], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=result/, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-203-g6371de8 Python-3.10.0 torch-1.12.1+cu102 CPU

Fusing layers... 
[W NNPACK.cpp:51] Could not initialize NNPACK! Reason: Unsupported hardware.
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
image 1/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1.x.000.y.000.jpg: 256x256 (no detections), 1691.5ms
image 2/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1.x.000.y.256.jpg: 256x256 (no detections), 1609.0ms
image 3/69468 /home/astra/projects/hackathon/omsk_chempion

image 57/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1001.x.512.y.512.jpg: 64x256 (no detections), 376.1ms
image 58/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1001.x.768.y.000.jpg: 256x256 (no detections), 1491.5ms
image 59/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1001.x.768.y.256.jpg: 256x256 (no detections), 1458.4ms
image 60/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1001.x.768.y.512.jpg: 64x256 (no detections), 370.0ms
image 61/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1002.x.000.y.000.jpg: 256x256 (no detections), 1565.7ms
image 62/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1002.x.000.y.256.jpg: 256x256 (no detections), 1528.4ms
image 63/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1002.x.000.y.512.jpg: 64x256 (no detections), 330.1ms
image 64/69468 /home/astra/projects/hackathon/o

image 118/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1006.x.768.y.000.jpg: 256x256 (no detections), 1509.0ms
image 119/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1006.x.768.y.256.jpg: 256x256 (no detections), 1549.1ms
image 120/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1006.x.768.y.512.jpg: 64x256 (no detections), 338.2ms
image 121/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1007.x.000.y.000.jpg: 256x256 (no detections), 1562.1ms
image 122/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1007.x.000.y.256.jpg: 256x256 (no detections), 1607.4ms
image 123/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1007.x.000.y.512.jpg: 64x256 (no detections), 324.9ms
image 124/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1007.x.256.y.000.jpg: 256x256 1 0, 1603.4ms
image 125/69468 /home/astra/projects/hackathon/oms

image 178/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1010.x.768.y.000.jpg: 256x256 (no detections), 1539.7ms
image 179/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1010.x.768.y.256.jpg: 256x256 1 0, 1510.6ms
image 180/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1010.x.768.y.512.jpg: 64x256 (no detections), 333.8ms
image 181/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1011.x.000.y.000.jpg: 256x256 (no detections), 1564.0ms
image 182/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1011.x.000.y.256.jpg: 256x256 (no detections), 1602.9ms
image 183/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1011.x.000.y.512.jpg: 64x256 (no detections), 371.5ms
image 184/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1011.x.256.y.000.jpg: 256x256 (no detections), 1543.4ms
image 185/69468 /home/astra/projects/hackathon/oms

image 239/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1015.x.768.y.256.jpg: 256x256 (no detections), 1598.7ms
image 240/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1015.x.768.y.512.jpg: 64x256 (no detections), 365.8ms
image 241/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1016.x.000.y.000.jpg: 256x256 (no detections), 1535.2ms
image 242/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1016.x.000.y.256.jpg: 256x256 (no detections), 1577.8ms
image 243/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1016.x.000.y.512.jpg: 64x256 (no detections), 419.9ms
image 244/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1016.x.256.y.000.jpg: 256x256 (no detections), 1577.0ms
image 245/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1016.x.256.y.256.jpg: 256x256 (no detections), 1573.8ms
image 246/69468 /home/astra/projects/h

...

image 1077/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1054.x.2048.y.2048.jpg: 256x256 (no detections), 1541.5ms
image 1078/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1054.x.2048.y.2304.jpg: 256x256 (no detections), 1500.2ms
image 1079/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1054.x.2048.y.256.jpg: 256x256 (no detections), 1544.2ms
image 1080/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1054.x.2048.y.2560.jpg: 256x256 (no detections), 1515.6ms
image 1081/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1054.x.2048.y.2816.jpg: 256x256 (no detections), 1518.9ms
image 1082/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1054.x.2048.y.3072.jpg: 32x256 (no detections), 158.6ms
image 1083/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1054.x.2048.y.512.jpg: 256x256 (no detections), 1494.9ms
image 1084/69468 

image 1136/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1054.x.2816.y.768.jpg: 256x256 (no detections), 1549.4ms
image 1137/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1054.x.3072.y.000.jpg: 256x256 (no detections), 1585.1ms
image 1138/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1054.x.3072.y.1024.jpg: 256x256 (no detections), 1573.2ms
image 1139/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1054.x.3072.y.1280.jpg: 256x256 (no detections), 1618.4ms
image 1140/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1054.x.3072.y.1536.jpg: 256x256 (no detections), 1559.3ms
image 1141/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1054.x.3072.y.1792.jpg: 256x256 (no detections), 1674.9ms
image 1142/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1054.x.3072.y.2048.jpg: 256x256 (no detections), 1646.1ms
image 1143/6946

image 1195/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1054.x.4096.y.2304.jpg: 256x256 (no detections), 1634.1ms
image 1196/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1054.x.4096.y.256.jpg: 256x256 (no detections), 1601.9ms
image 1197/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1054.x.4096.y.2560.jpg: 256x256 (no detections), 1682.3ms
image 1198/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1054.x.4096.y.2816.jpg: 256x256 (no detections), 1548.9ms
image 1199/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1054.x.4096.y.3072.jpg: 32x256 (no detections), 176.2ms
image 1200/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1054.x.4096.y.512.jpg: 256x256 (no detections), 1463.9ms
image 1201/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1054.x.4096.y.768.jpg: 256x256 (no detections), 1565.2ms
image 1202/69468 /

image 1254/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1054.x.5120.y.000.jpg: 256x256 (no detections), 1553.0ms
image 1255/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1054.x.5120.y.1024.jpg: 256x256 (no detections), 1675.4ms
image 1256/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1054.x.5120.y.1280.jpg: 256x256 (no detections), 1582.9ms
image 1257/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1054.x.5120.y.1536.jpg: 256x256 (no detections), 1706.6ms
image 1258/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1054.x.5120.y.1792.jpg: 256x256 (no detections), 1557.4ms
image 1259/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1054.x.5120.y.2048.jpg: 256x256 (no detections), 1483.5ms
image 1260/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1054.x.5120.y.2304.jpg: 256x256 (no detections), 1559.3ms
image 1261/694

image 1313/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1056.x.512.y.512.jpg: 64x256 (no detections), 333.6ms
image 1314/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1056.x.768.y.000.jpg: 256x256 (no detections), 1635.6ms
image 1315/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1056.x.768.y.256.jpg: 256x256 (no detections), 1786.2ms
image 1316/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1056.x.768.y.512.jpg: 64x256 (no detections), 316.0ms
image 1317/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1057.x.000.y.000.jpg: 256x256 (no detections), 1521.2ms
image 1318/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1057.x.000.y.256.jpg: 256x256 (no detections), 1634.7ms
image 1319/69468 /home/astra/projects/hackathon/omsk_chempionat/model-predict/crop/1057.x.000.y.512.jpg: 64x256 (no detections), 342.1ms
image 1320/69468 /home/astra/proj

In [7]:
def get_files(files_dir: Path, suff: str = 'png') -> list[Path]:
    return sorted(Path(files_dir).glob(f'*.{suff}'))

In [8]:
labels_list = get_files(labels, 'txt')

In [9]:
def str_to_float(labels_list: list[list[str]]) -> list[list[float]]:
    float_labels = []
    for label in labels_list:
        int_label = [float(x) for x in label]
        float_labels.append(int_label)
    return float_labels

In [10]:
def read_txt(label_file: Path) -> list[list[float]]:
    # функция возвращает список боксов с координатами в формате
    # [номер класса объекта] [x - центр] [y -центр] [ширина от центра] [высота от центра]
    # значения координат в диапазоне от 0 до 1 (доля от общих рамзеров картинки)
    with open(label_file, 'r') as labels:
        labels_list = labels.readlines()
        labels_list = [row.replace('\n', '') for row in labels_list]
        labels_list = [row.split(' ') for row in labels_list]
        return str_to_float(labels_list)
        

In [23]:
def fraction_to_pixel(labels: list[list[float]], x: int, y: int, size: tuple[int, int] = (256,256)) -> list[list[int]]:
    width, height = size
    #width = width + x
    #height = height + y
    new_labels = []
    for label in labels:
        _, x_center, y_center, obj_width, obj_height = label
        x_center = int(x_center * width + x)
        obj_width = obj_width * width
        y_center = int(y_center * height + y)
        obj_height = obj_height * height
        radius = int((obj_width + obj_height) / 2)
        new_labels.append([x_center, y_center, radius])
    return new_labels

In [28]:
def get_x_y(label_file: Path) -> tuple[int, int]:
    # .x.%03d.y.%03d.jpg
    name = label_file.stem
    name_list = name.split('.')
    parent_name = name_list[0]
    x_min = int(name_list[2])
    y_min = int(name_list[4])
    return parent_name, x_min, y_min

In [30]:
for label_file in labels_list:
    labels = read_txt(label_file)
    parent_name, x_min, y_min = get_x_y(label_file)
    pixel_labels = fraction_to_pixel(labels, x=x_min, y=y_min)
    real_labels = [parent_name]
    real_labels.extend(pixel_labels)
    print(real_labels)
    #break

['1000', [462, 185, 12]]
['1005', [506, 212, 5]]
['1005', [542, 248, 6]]
['1007', [311, 164, 7]]
['1010', [935, 268, 8]]
['1013', [140, 280, 6]]
['1013', [461, 249, 6]]
['1013', [289, 354, 4]]
['1014', [483, 95, 7]]
['1014', [952, 39, 12]]
['1015', [668, 506, 8]]
['1018', [572, 164, 9]]
['1019', [729, 340, 7]]
['1020', [516, 12, 6]]
['1021', [285, 444, 5]]
['1021', [1004, 8, 7]]
['1022', [160, 194, 6]]
['1024', [174, 321, 7]]
['1024', [446, 506, 10]]
['1026', [865, 108, 12]]
['1027', [928, 338, 5]]
['1040', [1796, 506, 9]]
['1040', [2050, 140, 9]]
['1040', [2287, 1276, 8]]
['1040', [2974, 1019, 10]]
['1040', [3419, 1531, 8]]
['1040', [3516, 761, 11]]
['1040', [4480, 505, 15]]
['1040', [4623, 1784, 16]]
['1042', [729, 259, 6]]
['1044', [76, 747, 19]]
['1045', [358, 326, 7], [431, 369, 8]]
['1047', [561, 388, 15]]
['1049', [377, 104, 9]]
['1049', [582, 98, 10]]
['1049', [990, 214, 10]]
['1049', [948, 284, 5]]
['106', [111, 508, 6], [99, 508, 4]]
['106', [61, 707, 18], [71, 747, 19]]
